In [ ]:
!sudo -H pip install sklearn

    100% |████████████████████████████████| 17.6MB 74kB/s
  Running setup.py bdist_wheel for sklearn ... - done
  Stored in directory: /root/.cache/pip/wheels/d7/db/a3/1b8041ab0be63b5c96c503df8e757cf205c2848cf9ef55f85e
Successfully built sklearn


In [ ]:
import cPickle as pickle
import re
import requests
from sklearn.feature_extraction.text import CountVectorizer

# Load Codelists

In [ ]:
sector_url = 'http://iatistandard.org/202/codelists/downloads/clv3/json/en/Sector.json'
sectors_json = requests.get(sector_url).json()['data']
sectors = {
    item['code']: item for item in sectors_json
}

In [ ]:
sectors

{u'11110': {u'category': u'111',
  u'code': u'11110',
  u'description': u'Education sector policy, planning and programmes; aid to education ministries, administration and management systems; institution capacity building and advice; school management and governance; curriculum and materials development; unspecified education activities.',
  u'name': u'Education policy and administrative management'},
 u'11120': {u'category': u'111',
  u'code': u'11120',
  u'description': u'Educational buildings, equipment, materials; subsidiary services to education (boarding facilities, staff housing); language training; colloquia, seminars, lectures, etc.',
  u'name': u'Education facilities and training'},
 u'11130': {u'category': u'111',
  u'code': u'11130',
  u'description': u'Teacher education (where the level of education is unspecified); in-service and pre-service training; materials development.',
  u'name': u'Teacher training'},
 u'11182': {u'category': u'111',
  u'code': u'11182',
  u'descri

In [ ]:
sector_categories_url = 'http://iatistandard.org/202/codelists/downloads/clv3/json/en/SectorCategory.json'
sector_categories_json = requests.get(sector_categories_url).json()['data']
sector_categories = {
    item['code']: item for item in sector_categories_json
}

In [ ]:
sector_categories

{u'111': {u'code': u'111',
  u'description': u'Education sector policy, planning and programmes; aid to education ministries, administration and management systems; institution capacity building and advice; school management and governance; curriculum and materials development; unspecified education activities.',
  u'name': u'Education, level unspecified'},
 u'112': {u'code': u'112',
  u'description': u'Formal and non-formal primary education for children; all elementary and first cycle systematic instruction; provision of learning materials.',
  u'name': u'Basic education'},
 u'113': {u'code': u'113',
  u'description': u'Second cycle systematic instruction at both junior and senior levels.',
  u'name': u'Secondary education'},
 u'114': {u'code': u'114',
  u'description': u'Degree and diploma programmes at universities, colleges and polytechnics; scholarships.',
  u'name': u'Post-secondary education'},
 u'121': {u'code': u'121',
  u'description': u'Health sector policy, planning and pr

In [ ]:
policy_marker_url = 'http://iatistandard.org/202/codelists/downloads/clv3/json/en/PolicyMarker.json'
policy_markers_json = requests.get(policy_marker_url).json()['data']
policy_markers = {
    item['code']: item for item in policy_markers_json
}

In [ ]:
policy_markers

{u'1': {u'code': u'1', u'name': u'Gender Equality'},
 u'2': {u'code': u'2', u'name': u'Aid to Environment'},
 u'3': {u'code': u'3', u'name': u'Participatory Development/Good Governance'},
 u'4': {u'code': u'4', u'name': u'Trade Development'},
 u'5': {u'code': u'5',
  u'name': u'Aid Targeting the Objectives of the Convention on Biological Diversity'},
 u'6': {u'code': u'6',
  u'name': u'Aid Targeting the Objectives of the Framework Convention on Climate Change - Mitigation'},
 u'7': {u'code': u'7',
  u'name': u'Aid Targeting the Objectives of the Framework Convention on Climate Change - Adaptation'},
 u'8': {u'code': u'8',
  u'name': u'Aid Targeting the Objectives of the Convention to Combat Desertification'},
 u'9': {u'code': u'9',
  u'name': u'Reproductive, Maternal, Newborn and Child Health (RMNCH)'}}

# Create Term Vectors

For the names and descriptions, we'll tokenize them and store the words that are used in the names/descriptions inside of a dictionary of sets. This will let us perform cosine similarity of search terms against the terms that are actually used in sector codes and policy markers.

In [ ]:
def add_attribute(item, item_attribute, search_item, search_attribute):
    if item_attribute not in item:
        search_item[search_attribute + '_raw'] = ''
        search_item[search_attribute] = set()
        return

    raw_value = item[item_attribute]

    if raw_value is None:
        search_item[search_attribute + '_raw'] = ''
        search_item[search_attribute] = set()
        return

    raw_value = raw_value.lower().strip()
    search_item[search_attribute + '_raw'] = raw_value

    if len(raw_value) == 0:
        search_item[search_attribute] = set()
        return

    tokenized_value = set([
        item for item in re.split('[^a-z]+', raw_value.strip().lower())
            if len(item) > 0
    ])

    search_item[search_attribute] = tokenized_value

In [ ]:
searchable_sectors = {}

for code, sector in sectors.iteritems():
    category_id = sector['category']

    sector_data = {}

    add_attribute(sector, 'name', sector_data, 'sector_name')
    add_attribute(sector, 'description', sector_data, 'sector_description')

    if category_id in sector_categories:
        category = sector_categories[category_id]
        add_attribute(category, 'name', sector_data, 'category_name')
        add_attribute(category, 'description', sector_data, 'category_description')
    else:
        print 'Missing category:', category_id

    searchable_sectors[code] = sector_data

Missing category: 231
Missing category: 231


In [ ]:
searchable_sectors

{u'11110': {'category_description': {u'activities',
   u'administration',
   u'advice',
   u'aid',
   u'and',
   u'building',
   u'capacity',
   u'curriculum',
   u'development',
   u'education',
   u'governance',
   u'institution',
   u'management',
   u'materials',
   u'ministries',
   u'planning',
   u'policy',
   u'programmes',
   u'school',
   u'sector',
   u'systems',
   u'to',
   u'unspecified'},
  'category_description_raw': u'education sector policy, planning and programmes; aid to education ministries, administration and management systems; institution capacity building and advice; school management and governance; curriculum and materials development; unspecified education activities.',
  'category_name': {u'education', u'level', u'unspecified'},
  'category_name_raw': u'education, level unspecified',
  'sector_description': {u'activities',
   u'administration',
   u'advice',
   u'aid',
   u'and',
   u'building',
   u'capacity',
   u'curriculum',
   u'development',
   u'educ

In [ ]:
searchable_policy_markers = {}

for code, policy_marker in policy_markers.iteritems():
    policy_marker_data = {}

    add_attribute(policy_marker, 'name', policy_marker_data, 'name')

    searchable_policy_markers[code] = policy_marker_data

In [ ]:
searchable_policy_markers

{u'1': {'name': {u'equality', u'gender'}, 'name_raw': u'gender equality'},
 u'2': {'name': {u'aid', u'environment', u'to'},
  'name_raw': u'aid to environment'},
 u'3': {'name': {u'development', u'good', u'governance', u'participatory'},
  'name_raw': u'participatory development/good governance'},
 u'4': {'name': {u'development', u'trade'}, 'name_raw': u'trade development'},
 u'5': {'name': {u'aid',
   u'biological',
   u'convention',
   u'diversity',
   u'objectives',
   u'of',
   u'on',
   u'targeting',
   u'the'},
  'name_raw': u'aid targeting the objectives of the convention on biological diversity'},
 u'6': {'name': {u'aid',
   u'change',
   u'climate',
   u'convention',
   u'framework',
   u'mitigation',
   u'objectives',
   u'of',
   u'on',
   u'targeting',
   u'the'},
  'name_raw': u'aid targeting the objectives of the framework convention on climate change - mitigation'},
 u'7': {'name': {u'adaptation',
   u'aid',
   u'change',
   u'climate',
   u'convention',
   u'framework',

# Save the Data

In [ ]:
with open('codelist_sectors.pickle', 'w') as f:
    pickle.dump(searchable_sectors, f)

In [ ]:
with open('codelist_policy_markers.pickle', 'w') as f:
    pickle.dump(searchable_policy_markers, f)

In [ ]:
!tar -cf codelist.tar codelist_*
!gzip codelist.tar
!aws s3 cp codelist.tar.gz s3://mdang.w210/ --acl public-read

upload: ./codelist.tar.gz to s3://mdang.w210/codelist.tar.gz
